In [7]:
%load_ext autoreload
%autoreload 2

import sys
import os
import inspect
import getpass
import json

import pandas as pd
import matplotlib.pyplot as plt


project_path = os.path.abspath( os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))) + "/../" )
sys.path.insert(0, project_path)
print(project_path)

from database_storage.helper import *

from database_storage.database.database import *
from database_storage.main import *

from database_storage.objects.human import *
from database_storage.objects.issue import *
from database_storage.objects.kpi import *
from database_storage.objects.machine import *
from database_storage.objects.tag import *
from database_storage.objects.maintenanceworkorder import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/Users/saschamoccozet/Documents/Git/Nestor


# Get CSVfile Path

### Greystone

In [ ]:
user = "neo4j"
password = 'GREYSTONE!!'

csv_file = os.path.join(project_path ,'data', 'gs_data', 'readable.csv')
csvSchemas_file = os.path.join(project_path , 'data',  'gs_data', 'gs_header.yaml')


bincsv_file = os.path.join(project_path ,'data', 'gs_data', 'bin_1g.csv')
binNcsv_file = os.path.join(project_path ,'data', 'gs_data', 'bin_Ng.csv')


### Mine

In [2]:
user = "neo4j"
password = 'MINE!!'

csv_file = os.path.join(project_path ,'data', 'mine_data', 'readable.csv')
csvSchemas_file = os.path.join(project_path , 'data',  'mine_data', 'mine_header.yaml')


bincsv_file = os.path.join(project_path ,'data', 'mine_data', 'bin_1g.csv')
binNcsv_file = os.path.join(project_path ,'data', 'mine_data', 'bin_Ng.csv')


In [4]:
tree_path = os.path.join(project_path ,'data', 'mine_data', 'tree_test.json')

### HVAC

In [ ]:
user = "neo4j"
password = 'HVAC!!'

csv_file = os.path.join(project_path ,'data', 'hvac_data', 'readable.csv')
csvSchemas_file = os.path.join(project_path , 'data',  'hvac_data', 'hvac_header.yaml')


bincsv_file = os.path.join(project_path ,'data', 'hvac_data', 'bin_1g.csv')
binNcsv_file = os.path.join(project_path ,'data', 'hvac_data', 'bin_Ng.csv')


### Others

In [ ]:
user = getpass.getuser()
password = getpass.getpass("Password:")

csv_file = os.path.join(project_path ,'data', 'hvac_data', 'hvac_raw.csv')
csvSchemas_file = os.path.join(project_path , 'data',  'hvac_data', 'hvac_header.yaml')

## CSV to Dataframe

In [5]:
dataframe = pd.read_csv(csv_file, index_col=0, header=0)
dataframe.fillna("", inplace=True)


dataframe_bincsv = pd.read_csv(bincsv_file, index_col=0, header=[0,1])
dataframe_binNcsv = pd.read_csv(binNcsv_file, index_col=0, header=[0,1])

csvSchemas_dict = openYAMLFile(csvSchemas_file)

yaml file open


In [8]:
with open(tree_path) as file:
    tree_dict = json.load(file)

# ConnectGraphDatabase

In [9]:
databaseSchema_file = os.path.join(project_path ,'database_storage','database', 'DatabaseSchema.yaml')
databaseSchema_dict = openYAMLFile(databaseSchema_file)
database = DatabaseNeo4J("bolt://127.0.0.1:7687", user, password, databaseSchema_dict)

yaml file open


## Clean the database

In [10]:
database.deleteData()

database.dropConstraints()
database.dropIndexes()

database.createIndexes()
database.createConstraints()

1

## Create Cypher Queries

### Create historical_data

In [11]:
queries = CypherCreate_historicalMaintenanceWorkOrder (database, dataframe, csvSchemas_dict)

100%|██████████| 5485/5485 [00:01<00:00, 4965.16it/s]


### Create tag_data

In [12]:
queries += CypherCreate_tag(database, dataframe_bincsv)

100%|██████████| 2058/2058 [01:07<00:00, 30.34it/s]


In [13]:
queries += CypherCreate_tag(database, dataframe_binNcsv)

100%|██████████| 1401/1401 [00:01<00:00, 1089.06it/s]


### Update 1gram-->Ngram

In [14]:
queries += CypherLink_Ngram1gram(database)

### Update issue-->item

In [15]:
queries += CypherLink_itemIssue(database)

### Update item-->item

In [16]:
queries += CypherCreate_itemsTree(database, tree_dict)

## Run Queries

In [17]:
database.runQueries(queries)

100%|██████████| 8965/8965 [00:48<00:00, 184.61it/s]


1